In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
# %matplotlib notebook
%matplotlib qt
def recoverTransformation(Image1, Image2, n):
    assert len(Image1.shape) == len(Image2.shape), 'Image1 and Image2 must have same channels'  
    
    # geting correspondences
    plt.imshow(Image1)
    image1_points = plt.ginput(n)
    plt.show()
    plt.close()

    plt.imshow(Image2)
    image2_points = plt.ginput(n)
    plt.close()
    # Setting up required equations
    X = np.array([[[pt[0], pt[1], 1, 0, 0, 0], [0, 0, 0, pt[0], pt[1], 1]] for pt in image2_points])
    c = X.shape[0]
    X = X.reshape(2*c,6)
    
    x_prime = []
    for pt in image1_points:
        x_prime.append(pt[0])
        x_prime.append(pt[1])
    x_prime = np.array(x_prime)
    
    # Recovering transformation
    X_pseudo = np.matmul(np.linalg.inv(np.matmul(np.transpose(X), X)), np.transpose(X))
    T = np.matmul(X_pseudo, x_prime)
    T = T.reshape(2,3)
    rows, cols, = Image1.shape[0], Image1.shape[1]
    T_Image = cv2.warpAffine(Image2,T,(cols,rows))
    
    A, t = T[:2, :2], [T[0,2], T[1,2]]
    T_points = []
    for p in image2_points:
        t_p = np.dot(A,p)+t
        T_points.append(t_p)
    # Calculating errors
    MSEPix = (np.square(np.array(image1_points) - np.array(T_points))).mean()
    MSECorPts = (np.square(Image1 - T_Image)).mean()
    return [MSEPix, MSECorPts, T, T_Image]

In [1]:
image1 = plt.imread('dataset/station1.png')
image2 = plt.imread('dataset/station2.png')
#image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
#image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
MSEPix, MSECorPts, T, T_Image = recoverTransformation(image1, image2, 5)
print ("The mean squared error of pixels of T_image and Image2 is" , MSEPix )
print("The mean squared error of correspondence points and transformed points is", MSECorPts)
print("Recovered transformation" , T)
cv2.imwrite('Results/Station_transformed_image.png', T_Image)
plt.imshow(T_Image)

NameError: name 'plt' is not defined